In [1]:
import alpaca_trade_api as tradeapi
import pandas as pd

api = tradeapi.REST(
    key_id='PKN37MG0JGVRM1DU9G7D',
    secret_key='UWeDq0aJiLKwH5iWE0fC8Mu4zX4wdQf342Vf8sD9',
    base_url='https://paper-api.alpaca.markets'
)

starting_cash = api.get_account().portfolio_value

TIMEZONE = 'America/New_York'

# API documentations
# https://docs.alpaca.markets/orders/
# https://github.com/alpacahq/alpaca-trade-api-python

In [2]:
import numpy as np
import pandas as pd
import time

In [3]:
params = {
    
    'window': 30, 
    'batch_size': 1,
    'epoch': 50,
    'num_features': 6,
    'patience': 3
}

In [4]:
df = pd.read_csv('2019_08_15_minute_data.csv', index_col = 0)

df['return'] = df['close'].pct_change()
df['return'].iloc[0] = 0

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(df, train_size = 0.7, shuffle = False)

scaler = MinMaxScaler(feature_range=(0, 1))
X_train_scaled = scaler.fit_transform(X_train.values)
X_test_scaled = scaler.transform(X_test.values)

/Users/munkarkin/environments/ml_dir/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [5]:
from keras.models import load_model
model = load_model('AAPL_min_LSTM.h5')

model.summary()

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 30, 50)            11400     
_________________________________________________________________
lstm_2 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 51        
Total params: 31,651
Trainable params: 31,651
Non-trainable params: 0
_________________________________________________________________


In [6]:

all_buys = []
all_sells = [] 


In [7]:

def sell_order(position):
    
    try:
    
        order = api.submit_order(

            symbol = position.symbol,
            qty = position.qty, 
            side = 'sell',
            type = 'market',
            time_in_force = 'day'
        )
        
        all_sells.append((api.get_order_by_client_order_id(order.client_order_id), order))
        
        print('Sell order SUCCESSFUL ...')
        
    
    except Exception as e:
        
        print('Sell order UNSUCCESSFUL, error: ' + e)
        
def buy_order(symbol, quantity):
    
    try:
    
        order = api.submit_order(

            symbol = symbol, 
            qty = quantity, 
            side = 'buy',
            type = 'market',
            time_in_force = 'day'
        )
        
        all_buys.append((api.get_order_by_client_order_id(order.client_order_id), order))
        
        print('Buy order SUCCESSFUL ...')
        
            
    except Exception as e:
        
        print('Buy order UNSUCCESSFUL, error: ' + e)
    

def step_1():
    # sell shares if purchased 
    
    positions = api.list_positions()
    
    for position in positions:
        # sell 
        sell_order(position)
    
def step_2():
    # access the last window of prices
    
    data = api.get_barset(
                ['AAPL'],
                'minute',
                limit = params['window'] + 1,
            ).df
    
    data = data['AAPL']
    data['return'] = data['close'].pct_change()
    
    # data.iloc[-params['window']:].values.reshape(1, params['window'], params['num_features']) 
    
    return scaler.transform(data)[-params['window']:].reshape((1, params['window'], params['num_features'])), data

def step_3(data):
    
    pred_array = np.zeros((1, 6))
    pred_array[0][3] = model.predict(data)[0][0]

    return scaler.inverse_transform(pred_array)[0][3]

def step_4(pred_close_next, close_price_now):
    
    '''
    Waiting is an important step since the orders aren't filled automatically,
    which means if your buys happen to come before your sells have filled,
    the buy orders will be bounced. In order to make the transition smooth,
    we sell first and wait for all the sell orders to fill before submitting
    our buy orders.
    '''
    
    # check for pending sell orders 
    
    while api.list_orders():
        time.sleep(0.1) # avoid making too many API calls 
    
    print('pred_close_next: {}, close_price_now: {}'.format(round(pred_close_next, 3), close_price_now))
    if pred_close_next > close_price_now:
        buy_order('AAPL', quantity = 20)
        

def track_account(time):
    
    print('Current time: {}'.format(time))
    
    print('Starting cash: {}'.format(starting_cash))
    print('Current portfolio value: {}'.format(api.get_account().portfolio_value))
    
    print('=================================================================================')
    print('=================================================================================\n\n')
    
    

In [ ]:
# initialize a time 
time_ref = pd.Timestamp.now(tz=TIMEZONE) - pd.Timedelta('1 min')

clock = api.get_clock()

# time = pd.Timestamp('09:30', tz=TIMEZONE) # (pd.Timestamp('09:30', tz=TIMEZONE) - pd.Timedelta('1 min'))

#time = clock.timestamp.tz_convert(TIMEZONE) - pd.Timedelta('1 days')

while clock.is_open:# and 1 minute has passed 
    
    # 1. sell shares if purchased
    # 2. access the last window of prices 
    # 3. model prediction for next time step 
    # 4. if prediction of next time step > current price, buy shares and sell it in the next time step. Buy only if
    #    there is no pending sell orders. 
    # 5. update time reference to make sure the code execute only once every minute   
    
    if not pd.Timestamp.now(tz=TIMEZONE).strftime('%H:%M') > time_ref.strftime('%H:%M'): # clock.timestamp.tz_convert(TIMEZONE).strftime('%H:%M')
        time.sleep(0.1)        
        continue 
    
    step_1()
    
    pred_data, data = step_2()
    
    #print(api.get_clock().timestamp)
    print('\n')
    print(data[['open', 'close']].iloc[-3:])
    print('\n')
    
    pred = step_3(pred_data)
    
    step_4(pred, data.iloc[-1].close)
    
    track_account(time_ref)
    
    clock = api.get_clock()

    time_ref += pd.Timedelta('1 min')



                              open   close
time                                      
2019-08-16 11:37:00-04:00  206.165  206.26
2019-08-16 11:38:00-04:00  206.250  206.26
2019-08-16 11:39:00-04:00  206.260  206.26


pred_close_next: 205.888, close_price_now: 206.26
Current time: 2019-08-16 11:40:09.439192-04:00
Starting cash: 100001.6
Current portfolio value: 100001.6




                             open   close
time                                     
2019-08-16 11:38:00-04:00  206.25  206.26
2019-08-16 11:39:00-04:00  206.26  206.26
2019-08-16 11:40:00-04:00  206.25  206.33


pred_close_next: 205.953, close_price_now: 206.33
Current time: 2019-08-16 11:41:09.439192-04:00
Starting cash: 100001.6
Current portfolio value: 100001.6




                             open   close
time                                     
2019-08-16 11:39:00-04:00  206.26  206.26
2019-08-16 11:40:00-04:00  206.25  206.33
2019-08-16 11:41:00-04:00  206.29  206.19


pred_close_next: 205.82, close_price_no

In [ ]:
now = pd.Timestamp.now(tz='Singapore')
end_dt = now
    
start_dt = end_dt - pd.Timedelta('1 days')
start = start_dt.strftime('%Y-%m-%d')
end = end_dt.strftime('%Y-%m-%d')

data = api.get_barset(
            ['AAPL'],
            'minute',
            start=start,
            end=end,
            limit = 1000
        ).df

In [ ]:
data['AAPL'].to_csv('latest_min_data.csv')